# Datasets 2015
Para cada año tenemos 3 tablas a inspeccionar, limpiar y unir:
- `title_basics`: pública y descargada de IMDb
- `title_ratings`: pública y descargada de IMDb
- `movies`: propia, escrapeada de la web de IMDb para todas las películas del año correspondiente

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
anno = 2015

In [3]:
# Tabla descargada de IMDb

title_basics = pd.read_csv('../data/imdb/title_basics.tsv', sep='\t', na_values='\\N')
print(title_basics.shape)
title_basics.head()

C:\Users\casiopa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(8084314, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short"


In [4]:
# Tabla descargada de IMDb

title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286


In [5]:
# Películas de escrapeadas de la web IMDb

movies = pd.read_csv('../data/web_imdb/movies_df_'+ str(anno) + '.csv', sep=';', na_values=["[]", "['']", "['', '']", "['', '', '']"])
print(movies.shape)
movies.head()

(16412, 20)


,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
0,tt0191476,Soapte de amor,NaN,2015.0,NaN,NaN,['Mircea Daneliuc'],['Mircea Daneliuc'],"['Horatiu Malaele', 'Ion Besoiu', 'Alexandru B...",['Comedy'],['Romania'],['Filmex'],5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0184221,Appunti per un film sul Jazz,NaN,2015.0,NaN,NaN,['Gianni Amico'],NaN,NaN,NaN,['Italy'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0297400,Snowblind,NaN,2015.0,NaN,NaN,NaN,NaN,NaN,"['Crime', 'Drama']","['United States', 'Canada']",['Moshag Productions'],6.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0283440,Kurzzeithelden,NaN,2015.0,NaN,NaN,"['Christine Wagner', 'Roman Gonther']","['Thomas F. Gallasch', 'Roman Gonther', 'Olaf ...","['Thomas Scharff', 'Francesco Pahlevan', 'Step...",['Sci-Fi'],['Germany'],['Wagner-Vision Production'],7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0337926,Chatô: O Rei do Brasil,NaN,2015.0,NaN,NaN,['Guilherme Fontes'],"['João Emanuel Carneiro', 'Guilherme Fontes', ...","['Marco Ricca', 'Andrea Beltrão', 'Paulo Betti']","['Biography', 'Drama']",['Brazil'],"['Estudios Quanta', 'GFF Filmes', 'ZB Faciliti...",6.0,NaN,NaN,Awards\n16 wins & 13 nominations,NaN,NaN,NaN,NaN


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15863 entries, 0 to 15862
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   imdb_id            15863 non-null  object 
 1   title              15862 non-null  object 
 2   original_title     623 non-null    object 
 3   year               15858 non-null  float64
 4   certificate        3429 non-null   object 
 5   duration           3409 non-null   object 
 6   directors          15530 non-null  object 
 7   writers            14504 non-null  object 
 8   stars              10902 non-null  object 
 9   genres             15371 non-null  object 
 10  countries          15524 non-null  object 
 11  companies          11017 non-null  object 
 12  imdb_rating        8908 non-null   float64
 13  metascore          871 non-null    float64
 14  popularity         160 non-null    object 
 15  awards             4078 non-null   object 
 16  budget             437

---
## Número de películas

En primer lugar verificar si hay una discrepancia entre el número de películas obtenidas de title_basics.tsv de IMDb
y el número de películas resultantes del scrapping.

In [6]:
# Cargar las películas de la base de datos de IMDb en una lista

imdb_ids = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]
imdb_ids = list(imdb_ids['tconst'])
len(imdb_ids)

16412

In [7]:
scrap_ids = list(movies['imdb_id'])
len(scrap_ids)

16412

---
## 1. Bd de IMDb: `title_basics`

### Variable `runtimeMinutes`  

In [9]:
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8084314 entries, 0 to 8084313
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   isAdult         float64
 5   startYear       float64
 6   endYear         float64
 7   runtimeMinutes  object 
 8   genres          object 
dtypes: float64(3), object(6)
memory usage: 555.1+ MB


#### Convertir `runtimeMinutes` en float
La siguiente función convierte los valores en float si se puede de manera natural, y si no se puede los imprime y convierte a nulo.

In [10]:
def to_float(n):
    try:
        n = float(n)
        return n
    except:
        print(n)
        return np.nan

In [11]:
title_basics.runtimeMinutes = title_basics.runtimeMinutes.apply(to_float)
title_basics.runtimeMinutes

Reality-TV
Documentary
Talk-Show
Game-Show
Reality-TV
Animation,Comedy,Family
Reality-TV


0           1.0
1           5.0
2           4.0
3          12.0
4           1.0
           ... 
8084309     NaN
8084310     NaN
8084311     NaN
8084312    27.0
8084313    10.0
Name: runtimeMinutes, Length: 8084314, dtype: float64

#### Eliminar de `title_basics` los registros con outliers en `runtimeMinutes`

In [12]:
# Buscar outliers mayores de 1.000 minutos y menores 15
# El resltado es un listado de películas fake, cortos o compendios de pelis.
# Eliminar todos estos registros del scrapping

title_basics = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]

title_basics = title_basics[~((title_basics.runtimeMinutes<15) | (title_basics.runtimeMinutes>500))]
title_basics.shape

(16407, 9)

In [13]:
# Una vez eliminados los outliers estos son los estadísticos:

title_basics.runtimeMinutes.describe().round(2)

count    13393.00
mean        88.40
std         25.79
min         15.00
25%         73.00
50%         88.00
75%        100.00
max        480.00
Name: runtimeMinutes, dtype: float64

---
## 2. Bd de IMDb: `title_ratings`

In [14]:
title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286



---
## 3. Bd propia escrapeada del portal IMDb: `movies`

## `year`
La base de datos de IMDb descargada tiene algunas discrepancias en cuanto al año de las películas, no coindice el año en la base de datos descargada de IMDb con el año del portal web, en el caso del año 2019 ocurre en 12 registros. Existe algún caso en el que tampoco coincide el año de la película en el portal Metacritic. Hay varias formas de afrontar estas discrepancias, en este caso, y por simplificación, utilizaré el año de la base de datos de IMDb.
Hay también algunos registros en la base de datos con valores nulos para el año y demás columnas, y serán eliminados (47 registors en total).

In [15]:
# Películas escrapeadas con año diferente
len(movies[(movies['year']!=anno) & movies['year'].notnull()])

4

In [16]:
movies[(movies['year']!=anno) & movies['year'].notnull()]

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
164,tt10659270,Rehab Cabin,NaN,2021.0,NaN,NaN,"['Kate Beacom', 'Louis Legge']","['Kate Beacom', ""Domenic D'Andrea""]","['Lacey Jeka', 'Scott Mandel', 'Alexandra Steb...",['Comedy'],['United States'],NaN,8.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2638,tt3130082,Pirate's Code: The Adventures of Mickey Matson,NaN,2014.0,Not Rated,1h 31min,['Harold Cronk'],"['Harold Cronk', 'Eric Machiela']","['Christopher Lloyd', 'Tia Carrere', 'Frank Co...",['Family'],['United States'],"['EMC Productions', 'Rebel Forge', '10 West St...",4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9947,tt4833838,Summer Issues,NaN,2020.0,NaN,NaN,['Eric Kelly'],"['Adam Masnyk', 'Rory Cardin']","['Vasilios Asimakos', 'Adam Masnyk', 'Matthew ...","['Comedy', 'Drama']",['United States'],['Another Theory Productions'],6.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11155,tt5005240,Not That Guy,NaN,2016.0,NaN,NaN,['Terry Traynor'],['Terry Traynor'],"['Matt Anderson', 'Mark Baillarageon', 'Robb B...","['Action', 'Adventure', 'Comedy']",['United States'],NaN,NaN,NaN,NaN,NaN,"$10,000",NaN,NaN,NaN


#### Eliminar del df escrapeado las películas de años posteriores a 2019
(son pelis futuras que se han colado)

In [17]:
movies = movies[~(movies['year']>2019)]
len(movies)

16410

#### Eliminar del df escrapeado las películas sin año
(son pelis sin información o url en el portal)

In [18]:
# Películas escrapeadas con año null, no tienen casi información, serán eliminadas
movies = movies[movies['year'].notnull()]
len(movies)

16403

## `imdb_rating`
Esta variable será sustituida por la de la base de datos `title_rating` descargada de IMDb

In [19]:
movies[['imdb_rating']].describe().round(2)

,imdb_rating
count,9076.00
mean,6.24
std,1.47
min,1.00
25%,5.40
50%,6.40
75%,7.30
max,10.00


## `metascore`
Comparing to: https://www.metacritic.com/feature/best-movies-released-in-2019

Hay pequeñas discrepancias en el listado de películas, debido a las fechas de estreno. Metacritic considera la fecha de estreno en USA. IMDb la fecha de estreno mundial. Pero sí que coinciden las puntuaciones y la mayoría de películas asignadas a 2019.

In [20]:
movies[['imdb_id', 'title', 'original_title', 'imdb_rating', 'metascore', 'year']][movies.metascore>=82].sort_values('metascore', ascending=False)

,imdb_id,title,original_title,imdb_rating,metascore,year
3622,tt3544082,45 años,45 Years,7.1,94.0,2015.0
1517,tt2096673,Del revés (Inside Out),Inside Out,8.1,94.0,2015.0
1822,tt2402927,Carol,NaN,7.2,94.0,2015.0
1393,tt1895587,Spotlight,NaN,8.1,93.0,2015.0
4286,tt3746250,Sherpa,NaN,7.6,93.0,2015.0
6762,tt4359416,Taxi Teherán,Taxi,7.3,91.0,2015.0
4511,tt3808342,El hijo de Saúl,Saul fia,7.5,91.0,2015.0
1049,tt1392190,Mad Max: Furia en la carretera,Mad Max: Fury Road,8.1,90.0,2015.0
6184,tt4238858,The Fits,NaN,6.6,90.0,2015.0
10847,tt4957446,Caminos del alma,Kang rinpoche,7.5,90.0,2015.0


In [21]:
np.sort(movies.metascore.dropna().unique())

array([ 5.,  9., 13., 14., 16., 17., 18., 19., 20., 21., 22., 23., 24.,
       25., 26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37.,
       38., 39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50.,
       51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63.,
       64., 65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76.,
       77., 78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89.,
       90., 91., 93., 94.])

In [22]:
movies.metascore.describe()

count    767.000000
mean      58.220339
std       17.701092
min        5.000000
25%       46.000000
50%       60.000000
75%       72.000000
max       94.000000
Name: metascore, dtype: float64

## `popularity`
Valor que vamos a descartar porque es muy volátil, cambia continuamente. Así, no guarda relación temporal con la taquilla que es el objeto de estudio.

In [23]:
# Convertirlo a número
movies['popularity'] = movies.popularity.str.replace(',', '').astype(float)

In [24]:
movies.popularity.describe()

count     141.000000
mean     2462.751773
std      1370.594611
min        98.000000
25%      1409.000000
50%      2460.000000
75%      3621.000000
max      4998.000000
Name: popularity, dtype: float64

## `directors`, `writers`, `stars`, `countries`, `companies`
Convertir a: comma separated values

In [25]:
movies['directors'] = movies['directors'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [26]:
movies['writers'] = movies['writers'].str.replace('[', '', regex=False) \
                                     .str.replace(']', '', regex=False) \
                                     .str.replace("'", "", regex=False) \
                                     .str.replace(", ", ",", regex=False)

In [27]:
movies['stars'] = movies['writers'].str.replace('[', '', regex=False) \
                                   .str.replace(']', '', regex=False) \
                                   .str.replace("'", "", regex=False) \
                                   .str.replace(", ", ",", regex=False)

In [28]:
movies['countries'] = movies['countries'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [29]:
movies['companies'] = movies['companies'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

## `opening_us_canada`, `gross_us_canada`
Convertir estos valores a números, eliminando el signo del dólar

In [30]:
def remove_dolar(amount):
    if str(amount) == 'nan':
        return np.nan
    else:        
        return float(amount.replace(',', '')[1:]) 

In [31]:
movies['gross_us_canada'] = movies['gross_us_canada'].apply(remove_dolar)
movies['gross_us_canada']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
16407   NaN
16408   NaN
16409   NaN
16410   NaN
16411   NaN
Name: gross_us_canada, Length: 16403, dtype: float64

In [32]:
movies['opening_us_canada'] = movies['opening_us_canada'].apply(remove_dolar)
movies['opening_us_canada']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
16407   NaN
16408   NaN
16409   NaN
16410   NaN
16411   NaN
Name: opening_us_canada, Length: 16403, dtype: float64

## `budget`
### Normalización `budget` a dólares
Son necesarios varios procesos para tener un valor válido de la variable `budget`:
- Separar, con regex, el texto que corresponde a la moneda y el que corresponde a la cantidad (ej. '€195,000'). Estos dos textos serán dos nuevas columnas `budgetCurrency` y `budgetAmount`
- Crear un diccionario con la correspondencia del código usado para las monedas en IMDb y el código ISO
- Crear un archivo externo que contenga las tasas de cambio para cada moneda y año
- Crear una nueva columna `budgetInDolars` con el valor ya normalizado a dólares

In [34]:
import re

def clean_amount_currency(raw_amount):
    
    if str(raw_amount) == 'nan' :
        return (np.nan, np.nan)
    
    # Diccionario con correspondencia del símbolo de moneda de IMDb y el ISO de monedas
    # Monedas de 2019 sin tasas de cambio: 'LVL'
    currency_to_iso = {'$':'USA', 'A$':'AUS', 'ZAR':'ZAF', '₹':'IND', '€':'EU27_2020',
                       'RUR':'RUS', 'R$':'BRA', 'CN¥':'CHN', 'TRL':'TUR', 'CA$':'CAN',
                       'NPR':'NPR', '£':'GBR', '¥':'JPN', 'HUF':'HUN', 'HRK':'HRV',
                       'NOK':'NOR', 'MNT':'MNT', 'CHF':'CHE', 'MX$':'MEX', 'UAH':'UAH',
                       'HK$':'HKG', 'PHP':'PHP', 'PLN':'POL', 'EGP':'EGP' , 'IRR':'IRR',
                       'NZ$':'NZL', 'AED':'AED', 'DKK':'DNK', 'NGN':'NGN', 'SEK':'SWE',
                       'BDT':'BDT', 'MYR':'MYR', 'CZK':'CZE', 'ARS':'ARG', 'PKR':'PKR',
                       'GEL':'GEL', 'NT$':'TWD', '₪':'ISR', '₫':'VND', '₩':'KOR', 'THB':'THB',
                       'RON':'ROU', 'PTE':'PRT', 'TTD':'TTD', 'MVR':'MVR', 'JOD':'JOD',
                       'ISK':'ISL', 'COP':'COL', 'LKR':'LKR', 'BSD':'BSD', 'BGL':'BGR',
                       'KZT':'KZT', 'KES':'KES', 'TZS':'TZS', 'DOP':'DOP', 'TJS':'TJS',
                       'TND':'TND', 'MUR':'MUR', 'SGD':'SGD', 'MOP':'MOP', 'IDR':'IDN',
                       'ETB':'ETB', 'CUP':'CUP', 'BYR':'BYR', 'MMK':'MMK', 'NAD':'NAD',
                       'PEN':'PER', 'MKD':'MKD', 'UGX':'UGX', 'BAM':'BAM', 'ALL':'ALL',
                       'KGS':'KGS', 'GHC':'GHS'
                      }
    
    raw_amount = raw_amount.replace(',', '')
    
    for item in re.finditer(r"^(\D+)(\d+)$", raw_amount):
        try:
            currency = currency_to_iso[item.group(1).strip()]
        except:
            currency = str(np.nan)
        amount = float(item.group(2))

    return(currency, amount)



# Dividir la info de budget en dos nuevas columnas (cantidad y moneda)
movies['budgetCurrency'], movies['budgetAmount'] = zip(*movies['budget'].map(clean_amount_currency))
movies[['title', 'budget', 'budgetAmount', 'budgetCurrency']].dropna()

# importar archivo csv de tasas de cambio
exchange_rates=pd.read_csv('../data/OECD/DP_LIVE_16072021155836489.csv')

# Seleccionar las tasas de cambio para el año del dataset
exchange_rates_anno = exchange_rates[exchange_rates.TIME == anno]

# Merge tasas de cambio con df movies, usando como primary key el código ISO de la moneda
movies = pd.merge(movies, exchange_rates_anno[['LOCATION','Value']], how='left', left_on='budgetCurrency', right_on='LOCATION')
movies['budgetInDollars'] = movies['budgetAmount'] / movies['Value']

movies.head()

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,...,awards,budget,gross_us_canada,opening_us_canada,gross_world,budgetCurrency,budgetAmount,LOCATION,Value,budgetInDollars
0,tt0191476,Soapte de amor,NaN,2015.0,NaN,NaN,Mircea Daneliuc,Mircea Daneliuc,Mircea Daneliuc,['Comedy'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0184221,Appunti per un film sul Jazz,NaN,2015.0,NaN,NaN,Gianni Amico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0297400,Snowblind,NaN,2015.0,NaN,NaN,NaN,NaN,NaN,"['Crime', 'Drama']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0283440,Kurzzeithelden,NaN,2015.0,NaN,NaN,"Christine Wagner,Roman Gonther","Thomas F. Gallasch,Roman Gonther,Olaf Lemitz","Thomas F. Gallasch,Roman Gonther,Olaf Lemitz",['Sci-Fi'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0337926,Chatô: O Rei do Brasil,NaN,2015.0,NaN,NaN,Guilherme Fontes,"João Emanuel Carneiro,Guilherme Fontes,Matthew...","João Emanuel Carneiro,Guilherme Fontes,Matthew...","['Biography', 'Drama']",...,Awards\n16 wins & 13 nominations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## `gross_world`
### Normalización `gross_world` a dólares
Para este año también es necesario normalizar a dólares esta variable.
Son necesarios varios procesos:
- Renombrar la columna a `gwNoDollars`
- Separar, con regex, el texto que corresponde a la moneda y el que corresponde a la cantidad (ej. '€195,000'). Estos dos textos serán dos nuevas columnas `gwCurrency` y `gwAmount`
- Crear un diccionario con la correspondencia del código usado para las monedas en IMDb y el código ISO
- Crear un archivo externo que contenga las tasas de cambio para cada moneda y año
- Crear una nueva columna `gross_world` con el valor ya normalizado a dólares

In [35]:
def print_rare(amount):
    if (str(amount) != 'nan') & (not str(amount).startswith('$')):
        print(amount)
    

movies['gross_world'].apply(print_rare);

₹320,000,000
₹660,100,000
₹287,000,000
₹740,300,000
PKR 157,500,000
₹600,000,000
₹500,500,000
₹550,000,000
₹200,000,000


In [36]:
movies.rename(columns={'gross_world': 'gwNoDollars'}, inplace=True)
movies.head(2)

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,...,awards,budget,gross_us_canada,opening_us_canada,gwNoDollars,budgetCurrency,budgetAmount,LOCATION,Value,budgetInDollars
0,tt0191476,Soapte de amor,NaN,2015.0,NaN,NaN,Mircea Daneliuc,Mircea Daneliuc,Mircea Daneliuc,['Comedy'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0184221,Appunti per un film sul Jazz,NaN,2015.0,NaN,NaN,Gianni Amico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Dividir la info de budget en dos nuevas columnas (cantidad y moneda)
movies['gwCurrency'], movies['gwAmount'] = zip(*movies['gwNoDollars'].map(clean_amount_currency))
#movies[['title', 'budget', 'budgetAmount', 'budgetCurrency']].dropna()

# importar archivo csv de tasas de cambio
exchange_rates=pd.read_csv('../data/OECD/DP_LIVE_16072021155836489.csv')

# Seleccionar las tasas de cambio para el año del dataset
exchange_rates_anno = exchange_rates[exchange_rates.TIME == anno]

# Merge tasas de cambio con df movies, usando como primary key el código ISO de la moneda
movies = pd.merge(movies, exchange_rates_anno[['LOCATION','Value']], how='left', left_on='gwCurrency', right_on='LOCATION', suffixes=('','_gw'))
movies['gross_world'] = movies['gwAmount'] / movies['Value_gw']

movies

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,...,budgetCurrency,budgetAmount,LOCATION,Value,budgetInDollars,gwCurrency,gwAmount,LOCATION_gw,Value_gw,gross_world
0,tt0191476,Soapte de amor,NaN,2015.0,NaN,NaN,Mircea Daneliuc,Mircea Daneliuc,Mircea Daneliuc,['Comedy'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0184221,Appunti per un film sul Jazz,NaN,2015.0,NaN,NaN,Gianni Amico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0297400,Snowblind,NaN,2015.0,NaN,NaN,NaN,NaN,NaN,"['Crime', 'Drama']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0283440,Kurzzeithelden,NaN,2015.0,NaN,NaN,"Christine Wagner,Roman Gonther","Thomas F. Gallasch,Roman Gonther,Olaf Lemitz","Thomas F. Gallasch,Roman Gonther,Olaf Lemitz",['Sci-Fi'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0337926,Chatô: O Rei do Brasil,NaN,2015.0,NaN,NaN,Guilherme Fontes,"João Emanuel Carneiro,Guilherme Fontes,Matthew...","João Emanuel Carneiro,Guilherme Fontes,Matthew...","['Biography', 'Drama']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16398,tt9894166,The Dance of Infinity,NaN,2015.0,NaN,NaN,Vladimir Maykov,Vladimir Maykov,Vladimir Maykov,['Documentary'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16399,tt9914828,The War of Godzilla,NaN,2015.0,NaN,NaN,Kyle,"Hiro,Jake,Willis","Hiro,Jake,Willis","['Action', 'Comedy', 'Family']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16400,tt9900680,Youtuber Horror Movie,NaN,2015.0,NaN,NaN,Marcus Butler,Marcus Butler,Marcus Butler,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16401,tt9902338,Fuitina Fuga D'Amore,NaN,2015.0,NaN,NaN,"Vito Trecarichi,Salvo Spoto","Salvo Spoto,Vito Trecarichi","Salvo Spoto,Vito Trecarichi",['Documentary'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
# Merge de tablas


In [38]:
imdb_merged = pd.merge(title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)] , title_ratings, on='tconst', how='left')
imdb_merged

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0184221,movie,Appunti per un film sul Jazz,Appunti per un film sul Jazz,0.0,2015.0,NaN,NaN,NaN,NaN,NaN
1,tt0191476,movie,Fed Up,Soapte de amor,0.0,2015.0,NaN,93.0,Comedy,5.9,120.0
2,tt0283440,movie,Short Time Heroes,Kurzzeithelden,0.0,2015.0,NaN,45.0,Sci-Fi,7.0,15.0
3,tt0297400,movie,Snowblind,Snowblind,0.0,2015.0,NaN,NaN,"Crime,Drama",6.7,77.0
4,tt0337926,movie,Chatô - The King of Brazil,Chatô: O Rei do Brasil,0.0,2015.0,NaN,102.0,"Biography,Drama",6.0,666.0
...,...,...,...,...,...,...,...,...,...,...,...
16402,tt9898548,movie,A Girl & the City,A Girl & the City,0.0,2015.0,NaN,60.0,Documentary,NaN,NaN
16403,tt9900680,movie,Youtuber Horror Movie,Youtuber Horror Movie,0.0,2015.0,NaN,NaN,NaN,NaN,NaN
16404,tt9902338,movie,Fuitina Fuga D'Amore,Fuitina Fuga D'Amore,0.0,2015.0,NaN,NaN,Documentary,NaN,NaN
16405,tt9914828,movie,The War of Godzilla,The War of Godzilla,0.0,2015.0,NaN,102.0,"Action,Comedy,Family",NaN,NaN


### Renombrar y seleccionar columnas de ambos datasets

In [39]:
imdb_merged.rename(columns={'primaryTitle': 'englishTitle',
                            'tconst': 'imdbId',
                            'averageRating': 'ratingImdb'
                          }, inplace=True)

In [40]:
imdb_merged.columns

Index(['imdbId', 'titleType', 'englishTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres', 'ratingImdb',
       'numVotes'],
      dtype='object')

In [41]:
movies.rename(columns={'imdb_id': 'imdbId',
                       'title': 'spanishTitle',
                       'budget': 'budgetString',
                       'budgetInDollars': 'budget',
                       'gross_us_canada': 'grossUsCanada',
                       'opening_us_canada': 'openingUsCanada',
                       'gross_world': 'grossWorld',
                      }, inplace=True)

In [42]:
movies.columns

Index(['imdbId', 'spanishTitle', 'original_title', 'year', 'certificate',
       'duration', 'directors', 'writers', 'stars', 'genres', 'countries',
       'companies', 'imdb_rating', 'metascore', 'popularity', 'awards',
       'budgetString', 'grossUsCanada', 'openingUsCanada', 'gwNoDollars',
       'budgetCurrency', 'budgetAmount', 'LOCATION', 'Value', 'budget',
       'gwCurrency', 'gwAmount', 'LOCATION_gw', 'Value_gw', 'grossWorld'],
      dtype='object')

In [43]:
# Selección de columnas de imdb
imdb_merged = imdb_merged[['imdbId', 'englishTitle', 'originalTitle', 'isAdult', 'runtimeMinutes', 'genres', 'ratingImdb', 'numVotes']]

In [44]:
# Selección de columnas del escrapeado
movies = movies[['imdbId', 'spanishTitle', 'year', 'certificate',
                'directors', 'writers', 'stars', 'countries', 'companies',
                'metascore', 'awards', 'budget', 'grossUsCanada',
                'openingUsCanada', 'grossWorld']]

### Merge final para este año

In [45]:
movies_merged = pd.merge(imdb_merged, movies, on='imdbId', how='inner')
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16398 entries, 0 to 16397
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           16398 non-null  object 
 1   englishTitle     16398 non-null  object 
 2   originalTitle    16398 non-null  object 
 3   isAdult          16398 non-null  float64
 4   runtimeMinutes   13389 non-null  float64
 5   genres           15729 non-null  object 
 6   ratingImdb       9065 non-null   float64
 7   numVotes         9065 non-null   float64
 8   spanishTitle     16398 non-null  object 
 9   year             16398 non-null  float64
 10  certificate      3228 non-null   object 
 11  directors        15767 non-null  object 
 12  writers          14405 non-null  object 
 13  stars            14405 non-null  object 
 14  countries        15869 non-null  object 
 15  companies        11443 non-null  object 
 16  metascore        767 non-null    float64
 17  awards      

In [46]:
# Reordenar columnas

movies_merged = movies_merged[['imdbId', 'year', 'spanishTitle', 'originalTitle',
                               'englishTitle', 'ratingImdb', 'numVotes', 'metascore',
                               'isAdult', 'certificate', 'runtimeMinutes', 'genres', 'directors',
                               'writers', 'stars', 'countries', 'companies', 'awards',
                               'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld']]

In [47]:
movies_merged

,imdbId,year,spanishTitle,originalTitle,englishTitle,ratingImdb,numVotes,metascore,isAdult,certificate,...,directors,writers,stars,countries,companies,awards,budget,grossUsCanada,openingUsCanada,grossWorld
0,tt0184221,2015.0,Appunti per un film sul Jazz,Appunti per un film sul Jazz,Appunti per un film sul Jazz,NaN,NaN,NaN,0.0,NaN,...,Gianni Amico,NaN,NaN,Italy,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0191476,2015.0,Soapte de amor,Soapte de amor,Fed Up,5.9,120.0,NaN,0.0,NaN,...,Mircea Daneliuc,Mircea Daneliuc,Mircea Daneliuc,Romania,Filmex,NaN,NaN,NaN,NaN,NaN
2,tt0283440,2015.0,Kurzzeithelden,Kurzzeithelden,Short Time Heroes,7.0,15.0,NaN,0.0,NaN,...,"Christine Wagner,Roman Gonther","Thomas F. Gallasch,Roman Gonther,Olaf Lemitz","Thomas F. Gallasch,Roman Gonther,Olaf Lemitz",Germany,Wagner-Vision Production,NaN,NaN,NaN,NaN,NaN
3,tt0297400,2015.0,Snowblind,Snowblind,Snowblind,6.7,77.0,NaN,0.0,NaN,...,NaN,NaN,NaN,"United States,Canada",Moshag Productions,NaN,NaN,NaN,NaN,NaN
4,tt0337926,2015.0,Chatô: O Rei do Brasil,Chatô: O Rei do Brasil,Chatô - The King of Brazil,6.0,666.0,NaN,0.0,NaN,...,Guilherme Fontes,"João Emanuel Carneiro,Guilherme Fontes,Matthew...","João Emanuel Carneiro,Guilherme Fontes,Matthew...",Brazil,"Estudios Quanta,GFF Filmes,ZB Facilities",Awards\n16 wins & 13 nominations,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16393,tt9898548,2015.0,A Girl & the City,A Girl & the City,A Girl & the City,NaN,NaN,NaN,0.0,NaN,...,Alina Reyzelman,Alina Reyzelman,Alina Reyzelman,Russia,NaN,NaN,10000.0,NaN,NaN,NaN
16394,tt9900680,2015.0,Youtuber Horror Movie,Youtuber Horror Movie,Youtuber Horror Movie,NaN,NaN,NaN,0.0,NaN,...,Marcus Butler,Marcus Butler,Marcus Butler,NaN,Wilderness Films,NaN,NaN,NaN,NaN,NaN
16395,tt9902338,2015.0,Fuitina Fuga D'Amore,Fuitina Fuga D'Amore,Fuitina Fuga D'Amore,NaN,NaN,NaN,0.0,NaN,...,"Vito Trecarichi,Salvo Spoto","Salvo Spoto,Vito Trecarichi","Salvo Spoto,Vito Trecarichi",Italy,Indipendent,NaN,NaN,NaN,NaN,NaN
16396,tt9914828,2015.0,The War of Godzilla,The War of Godzilla,The War of Godzilla,NaN,NaN,NaN,0.0,NaN,...,Kyle,"Hiro,Jake,Willis","Hiro,Jake,Willis",Canada,NaN,Awards\n1 win,NaN,NaN,NaN,NaN


## Feature engineering. Columnas `profit` y `roi`
Los estadísticos están distorsionados por los valores nulos que se eliminarán después

In [48]:
movies_merged['profit'] = movies_merged.grossWorld - movies_merged.budget
movies_merged['profit'].describe()

count    6.640000e+02
mean     2.882262e+07
std      1.410265e+08
min     -5.034794e+07
25%     -2.151676e+06
50%     -4.778090e+05
75%      3.794991e+06
max      1.824522e+09
Name: profit, dtype: float64

In [49]:
movies_merged['roi'] = (movies_merged.grossWorld - movies_merged.budget) / movies_merged.budget
movies_merged['roi'].describe()

count       664.000000
mean        864.551581
std       17728.271018
min          -0.999987
25%          -0.948731
50%          -0.607535
75%           0.879731
max      447802.372714
Name: roi, dtype: float64

---
# Inspección inicial de correlaciones
## Registros no nulos con los que hacer las correlaciones de valoraciones y recaudación
Valoraciones: imdb_rating, metascore

Recaudación: budget, gross_world


In [50]:
print('Registros con ratingImdb:', len(movies_merged[movies_merged.ratingImdb.notnull()]))
print('Registros con metascore:', len(movies_merged[movies_merged.metascore.notnull()]))

print('Registros con budget:', len(movies_merged[movies_merged.budget.notnull()]))
print('Registros con grossUsCanada:', len(movies_merged[movies_merged.grossUsCanada.notnull()]))
print('Registros con openingUsCanada:', len(movies_merged[movies_merged.openingUsCanada.notnull()]))
print('Registros con grossWorld:', len(movies_merged[movies_merged.grossWorld.notnull()]))

print('\nRegistros con las rating, metascore, budget, grossWorld:',len(movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]))

Registros con ratingImdb: 9065
Registros con metascore: 767
Registros con budget: 4564
Registros con grossUsCanada: 666
Registros con openingUsCanada: 607
Registros con grossWorld: 2411

Registros con las rating, metascore, budget, grossWorld: 285


In [51]:
movies_merged = movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285 entries, 6 to 12267
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           285 non-null    object 
 1   year             285 non-null    float64
 2   spanishTitle     285 non-null    object 
 3   originalTitle    285 non-null    object 
 4   englishTitle     285 non-null    object 
 5   ratingImdb       285 non-null    float64
 6   numVotes         285 non-null    float64
 7   metascore        285 non-null    float64
 8   isAdult          285 non-null    float64
 9   certificate      277 non-null    object 
 10  runtimeMinutes   285 non-null    float64
 11  genres           285 non-null    object 
 12  directors        285 non-null    object 
 13  writers          283 non-null    object 
 14  stars            283 non-null    object 
 15  countries        285 non-null    object 
 16  companies        285 non-null    object 
 17  awards        

## Correlaciones de valoraciones y recaudación

In [52]:
movies_merged.describe().round(2)

,year,ratingImdb,numVotes,metascore,isAdult,runtimeMinutes,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
count,285.0,285.00,285.00,285.00,285.0,285.00,2.850000e+02,2.430000e+02,2.360000e+02,2.850000e+02,2.850000e+02,285.00
mean,2015.0,6.43,83983.65,55.15,0.0,106.72,2.919267e+07,4.319577e+07,1.367134e+07,9.605237e+07,6.685970e+07,2.98
std,0.0,0.93,143244.91,18.01,0.0,15.56,4.498041e+07,9.663238e+07,3.112589e+07,2.425833e+08,2.090232e+08,25.57
min,2015.0,3.40,31.00,13.00,0.0,72.00,3.211567e+04,5.638000e+03,6.200000e+02,1.115000e+03,-5.034794e+07,-1.00
25%,2015.0,5.90,6128.00,41.00,0.0,96.00,3.200000e+06,1.981310e+05,3.343050e+04,9.338470e+05,-2.650801e+06,-0.82
50%,2015.0,6.60,32041.00,56.00,0.0,105.00,1.180000e+07,9.077245e+06,1.384282e+06,1.295109e+07,1.801241e+06,0.22
75%,2015.0,7.10,90639.00,69.00,0.0,118.00,3.400000e+07,4.363477e+07,1.305059e+07,7.979988e+07,4.677568e+07,2.36
max,2015.0,8.10,910725.00,94.00,0.0,168.00,2.500000e+08,9.366622e+08,2.479667e+08,2.069522e+09,1.824522e+09,428.64


In [53]:
method = 'pearson'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2015 Corr Método: pearson


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.717487,0.104702,0.162417,0.165963,-0.122100
metascore,0.717487,1.000000,0.012441,0.127517,0.145314,-0.066628
budget,0.104702,0.012441,1.000000,0.787205,0.698403,-0.017485
grossWorld,0.162417,0.127517,0.787205,1.000000,0.991155,0.039514
profit,0.165963,0.145314,0.698403,0.991155,1.000000,0.049621
roi,-0.122100,-0.066628,-0.017485,0.039514,0.049621,1.000000


In [54]:
method = 'kendall'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2015 Corr Método: kendall


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.530056,0.001035,0.099781,0.166888,0.161413
metascore,0.530056,1.000000,-0.133922,0.004162,0.142027,0.113119
budget,0.001035,-0.133922,1.000000,0.610890,0.237178,0.249448
grossWorld,0.099781,0.004162,0.610890,1.000000,0.628218,0.640425
profit,0.166888,0.142027,0.237178,0.628218,1.000000,0.639684
roi,0.161413,0.113119,0.249448,0.640425,0.639684,1.000000


In [55]:
method = 'spearman'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2015 Corr Método: spearman


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.703404,0.001427,0.146709,0.248101,0.232693
metascore,0.703404,1.000000,-0.200534,-0.000996,0.207028,0.163432
budget,0.001427,-0.200534,1.000000,0.793179,0.398181,0.360965
grossWorld,0.146709,-0.000996,0.793179,1.000000,0.795510,0.827116
profit,0.248101,0.207028,0.398181,0.795510,1.000000,0.849656
roi,0.232693,0.163432,0.360965,0.827116,0.849656,1.000000


In [56]:
movies_merged[['ratingImdb', 'metascore', 'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld', 'profit', 'roi']].corr()

,ratingImdb,metascore,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
ratingImdb,1.000000,0.717487,0.104702,0.153829,0.083623,0.162417,0.165963,-0.122100
metascore,0.717487,1.000000,0.012441,0.119045,0.059308,0.127517,0.145314,-0.066628
budget,0.104702,0.012441,1.000000,0.730037,0.731586,0.787205,0.698403,-0.017485
grossUsCanada,0.153829,0.119045,0.730037,1.000000,0.967797,0.957882,0.953972,0.035690
openingUsCanada,0.083623,0.059308,0.731586,0.967797,1.000000,0.956291,0.951646,0.041083
grossWorld,0.162417,0.127517,0.787205,0.957882,0.956291,1.000000,0.991155,0.039514
profit,0.165963,0.145314,0.698403,0.953972,0.951646,0.991155,1.000000,0.049621
roi,-0.122100,-0.066628,-0.017485,0.035690,0.041083,0.039514,0.049621,1.000000


---
# Guardado dataset a pickle

In [57]:
import pickle

with open('../data/web_imdb_clean/movies_2015.pickle', 'wb') as f:
    pickle.dump(movies_merged, f)